### Imports, Files, Functions and Constraints

In [102]:
import pandas as pd
from pathlib import Path
import numpy as np
import math
import scipy
from scipy.signal import find_peaks
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd

In [103]:
file_path_london_aadf = r"C:\Users\yusuf\Documents\TIL6022 Project\Data\dft_aadf_region_id_6.csv"                           # This is yearly count
file_path_london_aadf_direction = r"C:\Users\yusuf\Documents\TIL6022 Project\Data\dft_aadfbydirection_region_id_6.csv"      # This is yearly count by direction (2 ways included)
file_path_london_countpoints = r"C:\Users\yusuf\Documents\TIL6022 Project\Data\dft_countpoints_region_id_6.csv"             # This doesnt have any data, it is just the countpoint coordinates
file_path_london_rawcount = r"C:\Users\yusuf\Documents\TIL6022 Project\Data\dft_rawcount_region_id_6.csv"                   # This is hourly Count Rate (Too many data to process)
file_path_london_road_type = r"C:\Users\yusuf\Documents\TIL6022 Project\Data\dft_traffic_road_type_region_id_6.csv"         # This is the Road Class 
file_path_london_vehicletype = r"C:\Users\yusuf\Documents\TIL6022 Project\Data\dft_traffic_vehicle_type_region_id_6.csv"    # This is only for the whole london count

df_count = pd.read_csv(file_path_london_aadf)
df_count_per_side = pd.read_csv(file_path_london_aadf_direction)
df_countpoints = pd.read_csv(file_path_london_countpoints)
df_rawcounts = pd.read_csv(file_path_london_rawcount)
df_road_class = pd.read_csv(file_path_london_road_type)
df_london_count = pd.read_csv(file_path_london_vehicletype)

pd.set_option('display.max_columns', None)

In [104]:
intervals = [(2000, 2002), (2003, 2004), (2005, 2010), (2011, 2013), (2014, 2019), (2020, 2022)]

def map_to_interval(year):
    for interval in intervals:
        if interval[0] <= year <= interval[1]:
            return f"{interval[0]}-{interval[1]}"               # This is a function to get the intervals of congestion zone charge differences 

In [105]:
areas_inside_congestion_zone = ["Westminster", "Lambeth", "Southwark", "City of London",
                                 "Tower Hamlets", "Islington", "Camden"]                    # This is the general areas that are inside the congestion zone

roads_to_remove = ["A501", "A40", "A5", "A4202", "A4", "A302", "A3204", "A3", "A201", 
                   "Tower Bridge Rd", "A100", "A1210", "Great Eastern St"]                  #These are the main roads circling the congestion zone 

In [106]:
df_congestion = df_count[(df_count["local_authority_name"].isin(areas_inside_congestion_zone)) 
                         & (~df_count["road_name"].isin(roads_to_remove))]                    


In [107]:
df_yearly_car_type = df_congestion.groupby('year').agg({
    'pedal_cycles': 'sum',
    'two_wheeled_motor_vehicles': 'sum',
    'cars_and_taxis': 'sum',
    'buses_and_coaches': 'sum',
    'lgvs': 'sum',
    'all_hgvs': 'sum'
}).reset_index()

### Interval Count of all Motor Vehicles inside the Congestion Zone between change of prices 

In [108]:
df_yearly_congestion = df_congestion.groupby(["year"])["all_motor_vehicles"].sum().reset_index()

df_yearly_congestion['year_interval'] = df_yearly_congestion['year'].apply(map_to_interval)   

df_interval_congestion = df_yearly_congestion.groupby('year_interval')['all_motor_vehicles'].mean().reset_index()

fig = px.bar(df_interval_congestion, x='year_interval', y='all_motor_vehicles',
             labels={"all_motor_vehicles": "Average Total Motor Vehicles", "year_interval": "Year Interval"})

fig.show()


### Per Year Count Count of all Motor Vehicles inside the Congestion Zone

In [109]:
fig = px.bar(df_yearly_congestion, x="year", y="all_motor_vehicles", 
             labels={"all_motor_vehicles": "Total Motor Vehicles", "year": "Year"})

fig.show()

### Interval Car Type

In [110]:
df_yearly_car_type['year_interval'] = df_yearly_car_type['year'].apply(map_to_interval)

df_yearly_car_type

,year,pedal_cycles,two_wheeled_motor_vehicles,cars_and_taxis,buses_and_coaches,lgvs,all_hgvs,year_interval
0,2000,259440,508393,7853719,301823,1253869,408394,2000-2002
1,2001,283139,554560,7703744,294005,1278810,408481,2000-2002
2,2002,286073,526499,7462309,338625,1213620,360781,2000-2002
3,2003,353408,595809,7353649,383038,1321481,334309,2003-2004
4,2004,378420,607068,7075004,396905,1342749,323388,2003-2004
5,2005,484138,606873,6962340,386580,1395915,348907,2005-2010
6,2006,552433,664630,7033796,416908,1581480,345596,2005-2010
7,2007,503196,630610,6820125,414384,1629085,336366,2005-2010
8,2008,528269,603968,6770358,426049,1555995,354914,2005-2010
9,2009,539265,551645,6641734,423779,1409323,346930,2005-2010
